Mukta Mohindra Final project- 
Sentiment Analysis Twitter text

In [1]:
# setup
import sys
import subprocess
import pkg_resources
from collections import Counter
import re


required = {'spacy', 'scikit-learn', 'numpy', 
            'pandas', 'torch', 'matplotlib', 'wordcloud'}
 #           'transformers', 'allennlp==0.9.0'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

import spacy
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
#import transformers

from spacy.lang.en import English
!python -m spacy download en_core_web_md
import en_core_web_md
en = English()
nlp = en_core_web_md.load()





     |████████████████████████████████| 96.4MB 1.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=c064ab0e663fbb88735c4ff86bb83d94d39ee077fa884625763d744c3f7c21ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-xhbq7od8/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [2]:
with open('clean_tweets_10k.pkl', 'rb') as f:
    df_tweet = pickle.load(f)
print(df_tweet.head())
df_tweet.info()
print('Length',len(df_tweet))
##clean out any rows with null values

df_tweet.describe()
df_tweet['Target'].value_counts()

   Target                                               text
0       4  @gypsy_sunday :O omg oh yes, I always forget a...
1       4  at least i know @sarahtondryk and family will ...
2       0      Yesterday it was sunny and today its raining 
3       4  Going to see Chelsea Art College today.. shoul...
4       0  woken up 'early' 3days in a row and im exhaust...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Target  10000 non-null  int64 
 1   text    10000 non-null  object
dtypes: int64(1), object(1)
memory usage: 156.4+ KB
Length 10000


0    5014
4    4986
Name: Target, dtype: int64

In [3]:

#remove punctuation and URLs, and stopwords
def tokenize(text, model=nlp, nostopwds=True,  lemma=False):
   
    tokenlist = []
    doc = model(text)
    ent = ''
    for t in doc:
      
      if nostopwds and t.is_stop:
        #print(t.text)
        continue
      if t.like_url:
        tokenlist.append('URL')
        continue
      if not t.is_alpha:
        continue      
      if lemma:
        #print('lemma',text)
        tokenlist.append(t.lemma_)
      else:
        tokenlist.append(t.lower_)
    return tokenlist
text= "Lol, th? oh @you:all got &amp friend for the d?g ?.. U.S. I'm at a  buffet... Cine there got amore wat... "
print(tokenize(text,nostopwds=False))



['lol', 'th', 'oh', 'all', 'got', 'amp', 'friend', 'for', 'the', 'i', 'at', 'a', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']


#Let us do modeling based on words with log likelihood > 10 based on EDA

In [4]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle


In [8]:

X = df_tweet['text']
y = df_tweet['Target']




In [9]:
log_words=['want','work','thanks','miss', 'love','new','good','lol']
cv = CountVectorizer(tokenizer=tokenize, vocabulary=log_words)
vec_cv = cv.fit_transform(df_tweet['text']).toarray()
vec_cv.shape


(10000, 8)

In [10]:


svc = LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(vec_cv, y, 
                                                        test_size=.3, 
                                                        random_state=101)
svc.fit(X_train,y_train)

  

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [11]:
predict_train = svc.predict(X_train)
predict_test = svc.predict(X_test)
print('Train Accuracy Score is ',metrics.accuracy_score(y_train, predict_train))
print('Test Accuracy Score is ',metrics.accuracy_score(y_test, predict_test))

Train Accuracy Score is  0.5625714285714286
Test Accuracy Score is  0.569


#That is very low accuracy, let us model based on the complete tfidf model. Trying 3 models SVC, LogisticRegression, RandomForest

In [12]:
df_tweet = shuffle(df_tweet)
X = df_tweet['text']
y = df_tweet['Target']

tf = TfidfVectorizer(tokenizer=tokenize)
tf_vec = tf.fit_transform(df_tweet['text'])




In [13]:
X_train, X_test, y_train, y_test = train_test_split(tf_vec,y, test_size=.3, random_state=53)
print(X_train.shape, X_test.shape)

(7000, 12558) (3000, 12558)


confusion matrix explanation https://www.geeksforgeeks.org/confusion-matrix-machine-learning/

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix


def model_predict(X_train,y_train,X_test, y_test, modeltype='SVM'):

  if modeltype =='SVM':
     model = LinearSVC()
  elif modeltype == 'LOGREG':
     model = LogisticRegression()
  else:
     model = RandomForestClassifier()
  model.fit(X_train,y_train)
  predict_train = model.predict(X_train)
  predict_test = model.predict(X_test)
  print('Train Accuracy Score ',modeltype, metrics.accuracy_score(y_train, predict_train))
  print('Test Accuracy Score ',modeltype, metrics.accuracy_score(y_test, predict_test))
  print('confusion matrix\n',confusion_matrix(y_test, predict_test))

In [15]:
#model_predict(tf_vec_train,y_train,tf_vec_test, y_test,'SVM')
model_predict(X_train,y_train,X_test, y_test,'SVM')

Train Accuracy Score  SVM 0.9577142857142857
Test Accuracy Score  SVM 0.703
confusion matrix
 [[1063  478]
 [ 413 1046]]


In [16]:
model_predict(X_train,y_train,X_test, y_test,'LOGREG')

Train Accuracy Score  LOGREG 0.8737142857142857
Test Accuracy Score  LOGREG 0.7266666666666667
confusion matrix
 [[1083  458]
 [ 362 1097]]


In [17]:
model_predict(X_train,y_train,X_test, y_test,'RANDOM')



Train Accuracy Score  RANDOM 0.9938571428571429
Test Accuracy Score  RANDOM 0.7073333333333334
confusion matrix
 [[1050  491]
 [ 387 1072]]


The train accuracy is high but the test accuracy is very low usually means overfitting.

Let us build a pipeline and save

In [18]:
# output simple review-based models for deployment 
from sklearn.pipeline import Pipeline
tf = TfidfVectorizer(lowercase=False, min_df=0.01)
vec_tf = tf.fit_transform(df_tweet['text'])
df_tweet['Target'].replace(4,1, inplace=True)
df_tweet['Target'].value_counts()
y = df_tweet['Target'].to_numpy()
svc= LinearSVC()
svc = svc.fit(vec_tf,y )
pipe = Pipeline(steps=[('tfidf', tf), ('svc', svc)])
print('test pipe:', pipe.predict([df_tweet.iloc[7]['text']]))
pickle.dump(pipe, open('basicmodel_pipe.pkl', 'wb'))

test pipe: [0]
